# Embedding应用

In [ ]:
from FlagEmbedding import FlagModel

sentences_1 = ["这个周末我计划去海边度假，享受阳光和海浪", "最新研究表明，定期运动有助于提高工作效率和创造力"]
sentences_2 = ["我期待的假期是在沙滩上，听着海浪声放松", "科技公司最近发布了一款新的智能手机，引起了广泛关注"]

model = FlagModel("/root/autodl-tmp/bge-large-zh-v1.5", use_fp16=True)

#model = FlagModel("BAAI/bge-large-zh-v1.5", use_fp16=True)

embeddings_1 = model.encode(sentences_1)
print(embeddings_1.shape)

embeddings_2 = model.encode(sentences_2)
print(embeddings_2.shape)
print("\n\n")

# @运算符计算两组嵌入之间的点积
similarity = embeddings_1 @ embeddings_2.T

print(similarity)


## Embedding应用
### Embedding微调
数据格式


`{"query": str, "pos": List[str], "neg":List[str]}`  

`query` 是查询，`pos `是正样本文本列表，`neg `是负样本文本列表。如果查询没有负样本文本，可以从整个语料库中随机采样一些文本作为负样本文本。  

**样本示例**   

```shell
{
  "query": "一只黄色的狗沿着森林小径跑。",
  "pos": ["一只狗正在跑"],
  "neg": ["一只猫在跑", "Steele没有保持她最初的故事。", "这条规则阻止人们支付他们的子女抚养费。", "一个穿着背心的男人坐在车里。", "穿着黑色衣服，戴着白色头巾和太阳镜的人在公交车站等待。", "无论是Globe还是Mail都没有对加拿大道路系统的当前状态发表评论。", "Spring Creek设施旧且过时。"]
}

{
  "query": "它规定了每个阶段的基本活动以及与这些活动相关的关键因素。",
  "pos": ["对基本活动的关键因素进行了规定。"],
  "neg": ["它规定了关键活动，但没有为与这些活动相关的关键因素提供规定。", "人们聚集在一起抗议。", "州政府更希望你这么做。", "一个女孩坐在一个男孩旁边。", "两个男性正在表演。", "没有人在跳跃", "有人密谋攻击Conrad，击中他的头部。"]
}
```

难负例是一种广泛使用的方法，用于提高句子嵌入的质量。通过使用难负例，系统可以更好地学习区分相似但不完全相同的句子，从而提高模型在特定任务上的性能，比如文本相似性判断或句子分类。难负例通常是与正例（即目标句子）在表面上相似但语义上不同的句子，使用这些难负例可以帮助模型学习到更加鲁棒的句子表示。   

**基于FlagEmbedding框架生成难负例样本**  

```shell
python -m FlagEmbedding.baai_general_embedding.finetune.hn_mine \
--model_name_or_path /slurm/resources/weights/huggingface/BAAI/bge-large-zh-v1.5 \
--input_file ./dataset.jsonl \
--output_file ./dataset_minedHN.jsonl \
--range_for_sampling 2-200 \
--negative_number 8
```  

**input_file：**用于微调的json数据。该脚本将为每个查询检索前k个文档，并从前k个文档中随机抽样负例（不包括正面文档）。  
**output_file：**保存用于微调的含有挖掘到的难负例的JSON数据的路径  
**negative_number：**抽样负例的数量  
**range_for_sampling：**在哪里抽样负例。例如，2-100意味着从前2到前200个文档中抽样negative_number个负例。你可以设置更大的值来降低负例的难度（例如，设置为60-300，从前60到前300个段落中抽样负例）  
**candidate_pool：**检索的池。默认值为None，此脚本将从input_file中所有neg的组合中检索。该文件的格式与预训练数据相同。如果输入一个candidate_pool，则此脚本将从该文件中检索负例。  
**use_gpu_for_searching：**是否使用faiss-gpu来检索负例。

## 开始训练

**1. 多块GPU的情况**  

```shell
torchrun --nproc_per_node {number of gpus} \
-m FlagEmbedding.baai_general_embedding.finetune.run \
--output_dir {path to save model} \
--model_name_or_path BAAI/bge-large-zh-v1.5 \
--train_data ./toy_finetune_data.jsonl \
--learning_rate 1e-5 \
--fp16 \
--num_train_epochs 5 \
--per_device_train_batch_size {large batch size; set 1 for toy data} \
--dataloader_drop_last True \
--normlized True \
--temperature 0.02 \
--query_max_len 64 \
--passage_max_len 256 \
--train_group_size 2 \
--negatives_cross_device \
--logging_steps 10 \
--query_instruction_for_retrieval "" 
```

**2.单块GPU的情况**  

```shell
python -m FlagEmbedding.baai_general_embedding.finetune.run \
--output_dir ./output_finetuned_model \
--model_name_or_path /slurm/resources/weights/huggingface/BAAI/bge-large-zh-v1.5 \
--train_data ./dataset.jsonl \
--learning_rate 1e-5 \
--fp16 \
--num_train_epochs 5 \
--per_device_train_batch_size 1 \
--dataloader_drop_last True \
--normlized True \
--temperature 0.02 \
--query_max_len 64 \
--passage_max_len 256 \
--train_group_size 2 \
--logging_steps 10 \
--query_instruction_for_retrieval "" 
```

**per_device_train_batch_size:** 训练时每个设备的批量大小。在大多数情况下，更大的批量大小将带来更强的性能。可以通过启用`--fp16`、`--deepspeed ./df_config.json`（可以参考ds_config.json）、`--gradient_checkpointing`等来扩展它。  

**train_group_size:** 训练中每个查询的正样本和负样本数量。总是有一个正样本，所以这个参数将控制负样本的数量（#negatives=train_group_size-1）。注意负样本的数量不应该大于数据中"neg":List[str]的负样本数量。除了这个组中的负样本外，在微调中也会使用批内负样本。  

**negatives_cross_device:** 在所有GPU之间共享负样本。这个参数将扩展负样本的数量。  

**learning_rate:** 学习率，推荐使用1e-5/2e-5/3e-5。  

**temperature:** 它将影响相似度分数的分布。  

**query_max_len:** 查询的最大长度。请根据你的数据中查询的平均长度来设置。  

**passage_max_len:** 文章的最大长度。请根据你的数据中文章的平均长度来设置。  

**query_instruction_for_retrieval:** 查询的指令，将被添加到每个查询中。也可以设置为""以不添加任何内容到查询中。  

**use_inbatch_neg:** 使用同一批次中的文章作为负样本。默认值为True。

## 基于 LM-Cocktail 进行模型融合【可选操作】
微调基础bge模型可以提高其在目标任务上的性能，但可能会导致模型在目标领域之外的通用能力严重退化（例如，在c-mteb任务上的性能下降）。通过合并微调模型和基础模型，LM-Cocktail能够显著提升下游任务的性能，同时保持在其他无关任务上的性能。


In [1]:
from LM_Cocktail import mix_models, mix_models_with_data

# 合并 fine-tuned model 和 base model
# 注意：修改为你自己的模型路径、输出路径
model = mix_models(
    model_names_or_paths=["/slurm/resources/weights/huggingface/BAAI/bge-large-zh-v1.5", "output_finetuned_model"], 
    model_type='encoder', 
    weights=[0.5, 0.5],  # 可以改变权重以获得更好的权衡
    output_path='./output_finetuned_model1')

/slurm/home/admin/.conda/envs/dl/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


loading /slurm/resources/weights/huggingface/BAAI/bge-large-zh-v1.5 -----------------
loading output_finetuned_model -----------------
***weight for each model***: 
/slurm/resources/weights/huggingface/BAAI/bge-large-zh-v1.5 0.5
output_finetuned_model 0.5
Saving the new model to ./output_finetuned_model1
Transform the model to the format of 'sentence_transformers' (pooling_method='cls', normalized=True)


## 加载微调后的模型进行测试


In [1]:
from FlagEmbedding import FlagModel
sentences_1 = ["这个周末我计划去海边度假，享受阳光和海浪", "最新研究表明，定期运动有助于提高工作效率和创造力"]
sentences_2 = ["我期待的假期是在沙滩上，听着海浪声放松", "科技公司最近发布了一款新的智能手机，引起了广泛关注"]

model = FlagModel("./output_finetuned_model1", use_fp16=True)
#model = FlagModel("/root/autodl-tmp/output_mixed_model", use_fp16=True)

embeddings_1 = model.encode(sentences_1)
embeddings_2 = model.encode(sentences_2)

# @运算符计算两组嵌入之间的点积
similarity = embeddings_1 @ embeddings_2.T

print(similarity)

/slurm/home/admin/.conda/envs/dl/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[[0.669  0.2168]
 [0.2954 0.3157]]


## embedding模型评估

**评估命令:**  
```shell
python -m FlagEmbedding.baai_general_embedding.finetune.eval_msmarco \
--encoder /root/autodl-tmp/output_mixed_model \
--fp16 \
--add_instruction \
--k 10
```   


**encoder:**  指定编码器模型，可以是huggingface上的模型或本地模型。  
**fp16:** 使用半精度进行推断。  
**add_instruction:** 添加检索指令  
**k:** 指定为每个查询检索多少个最近邻居。  

